In [16]:
import os, sys, argparse, importlib, time, inspect, json
import numpy as np
import matplotlib.pyplot as plt
import os.path as osp
if hasattr(__builtins__,'__IPYTHON__'):
    print('Notebook')
    from tqdm.notebook import tqdm
else:
    print('Not notebook')
    from tqdm import tqdm
from tensorflow.keras import backend as K
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
import tensorflow_probability as tfp
import pandas as pd

# gpu_devices = tf.config.list_physical_devices('GPU') 
# if len(gpu_devices) > 0:
#     print("GPU detected")
#     tf.config.experimental.set_memory_growth(gpu_devices[0], True)
# else:
#     print('No GPU detected')

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.models import load_model, Model
import spektral
from sklearn.preprocessing import normalize
from spektral.data import DisjointLoader, BatchLoader, SingleLoader
from importlib import reload
import dev.data_load as dl
reload(dl)
graph_data=dl.graph_data

Notebook
Not notebook


In [2]:
os.getcwd()

'/lustre/hpc/hep/chri862z/work/IceCube/from_config'

In [3]:
os.chdir("..")

In [4]:
os.system("python submit_results.py -list_databases TRUE")

0

In [5]:
with tf.device('/cpu:0'): # if on the cpu
    model=tf.keras.models.load_model('from_config/trained_models/IceCube/Sage_sage1nonorm_10_2aauycmh')
    model.compile()
# batch_size=512
# dataset=graph_data(n_data=100000,skip=int(1.4e6-1), restart=1, transform=True, angle=True, unitvec=False)
# loader = DisjointLoader(dataset, batch_size=batch_size, epochs=1)

In [10]:
batch_size=512
dataset=graph_data(n_data=100000,skip=int(1.4e6-1), restart=1, transform=True, angle=True, unitvec=False, transform_path='db_files/muongun/transformers.pkl', db_path= 'db_files/muongun/rasmus_classification_muon_3neutrino_3mio.db')
loader = DisjointLoader(dataset, batch_size=batch_size, epochs=1)

Removed and ready to reload
Connecting to db-file
Loading Muons
Reading files
/groups/hep/chri862z/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.preprocessing.data module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)
/groups/hep/chri862z/anaconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator RobustScaler from version 0.19.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
  0%|          | 0/100000 [00:00<?, ?it/s]Splitting data to events
     dom_x    dom_y    dom_z      time  charge_log10  SRTInIcePulses
0  0.13203  0.20298  0.12828 -0.060393     -1.921554               1
1  0

In [11]:
@tf.function(input_signature = loader.tf_signature(), experimental_relax_shapes = True)
def test_step(inputs, targets):
    predictions = model(inputs, training = False)
    targets     = tf.cast(targets, tf.float32) 

    return predictions, targets


In [12]:
def predict(loader):
    prediction_list, target_list = [], []
    for batch in loader:
        inputs, targets = batch
        predictions, targets = test_step(inputs, targets)

        prediction_list.append(predictions.numpy())
        target_list.append(targets.numpy())
        y_reco  = tf.concat(prediction_list, axis = 0).numpy()
        y_true  = tf.concat(target_list, axis = 0)
        y_true  = tf.cast(y_true, tf.float32).numpy()
    return y_reco, y_true

In [13]:
reco, true=predict(loader)

In [18]:
reco_str=['energy_log10_pred', 'zenith_pred', 'azimuth_pred', 'zenith_sigma', 'azimuth_sigma']
recos=pd.DataFrame(reco)
recos.columns=reco_str
recos.head() ### change azi/zeni sigma to real sig

,energy_log10_pred,zenith_pred,azimuth_pred,zenith_sigma,azimuth_sigma
0,2.442123,0.459739,0.154261,11.903859,131.819977
1,2.465787,0.378621,0.698523,0.219347,30.790274
2,2.509951,0.491649,1.775593,26.031609,157.727448
3,2.299295,0.639824,2.637994,3.692753,21.571054
4,2.684694,0.775960,2.771619,15.115727,54.793076


In [ ]:
### get event_nos through data_load (just make an extra file with event_nos)